## Homework: Search Evaluation

In this homework, we will evaluate the results of vector
search.

> It's possible that your answers won't match exactly. If it's the case, select t

In [1]:
# !pip install -U minsearch qdrant_client

## Evaluation data

For this homework, we will use the same dataset we generated
in the videos.

In [2]:
# Load dataset 
import requests
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')

Here, `documents` contains the documents from the FAQ database
with unique IDs, and `ground_truth` contains generated
question-answer pairs. 

Also, we will need the code for evaluating retrieval:

In [3]:
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

/Users/aidenerdogan/Desktop/dev/learning/LLM-Zoomcamp/llmZoomCampEnv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Q1. Minsearch text

Now let's evaluate our usual minsearch approach, indexing documents with:

In [4]:
from minsearch import Index


index = Index(
    text_fields=["question", "section", "text"],
    keyword_fields=["course", "id"]
)

index.fit(documents)

In [5]:
boost = {'question': 1.5, 'section': 0.1}

def search_function(q):
    return index.search(q['question'], boost_dict=boost)

In [6]:
result = evaluate(ground_truth, search_function)
print(result)

100%|██████████| 4627/4627 [00:06<00:00, 752.82it/s]

{'hit_rate': 0.8597363302355738, 'mrr': 0.6897542375497872}


## Embeddings 

The latest version of minsearch also supports vector search. 
We will use it:

We will also use TF-IDF and Singular Value Decomposition to 
create embeddings from texts. You can refer to our
["Create Your Own Search Engine" workshop](https://github.com/alexeygrigorev/build-your-own-search-engine)
if you want to know more about it.

In [7]:
from minsearch import VectorSearch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

In [8]:
# Let's create embeddings for the "question" field:

texts = [doc["question"] for doc in documents]

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
X = pipeline.fit_transform(texts)

## Q2. Vector search for question

Now let's index these embeddings with minsearch:

In [9]:
vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)

In [10]:
# search function for vector search
def search_function(q):
    vec = pipeline.transform([q["question"]])[0]
    return vindex.search(vec)

In [11]:
# result
result = evaluate(ground_truth, search_function)
print(result)

100%|██████████| 4627/4627 [00:02<00:00, 1732.91it/s]

{'hit_rate': 0.4696347525394424, 'mrr': 0.30029588234688703}


## Q3. Vector search for question and answer

We only used question in Q2. We can use both question and answer:


In [12]:
texts = [doc["question"] + " " + doc["text"] for doc in documents]

In [13]:
from minsearch import VectorSearch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)

X = pipeline.fit_transform(texts)

In [14]:
vindex = VectorSearch(keyword_fields={"course"})
vindex.fit(X, documents)

In [15]:
def search_function(q):
    text = q["question"]
    vec = pipeline.transform([text])[0]
    return vindex.search(vec)

In [16]:
result = evaluate(ground_truth, search_function)
print(result)

100%|██████████| 4627/4627 [00:03<00:00, 1227.55it/s]

{'hit_rate': 0.8415820185865571, 'mrr': 0.6254320739894556}


## Q4. Qdrant

Now let's evaluate the following settings in Qdrant:

- `text = doc['question'] + ' ' + doc['text']`
- `model_handle = "jinaai/jina-embeddings-v2-small-en"`
- `limit = 5`

What's the MRR?

In [18]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("jinaai/jina-embeddings-v2-small-en")

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.3.intermediate.den

In [19]:
texts = [doc["question"] + " " + doc["text"] for doc in documents]
vectors = model.encode(texts, show_progress_bar=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]/Users/aidenerdogan/Desktop/dev/learning/LLM-Zoomcamp/llmZoomCampEnv/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
Batches: 100%|██████████| 30/30 [00:05<00:00,  5.56it/s]


In [20]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

# Initialize in-memory Qdrant
client = QdrantClient(":memory:", prefer_grpc=False)

# Create collection
client.recreate_collection(
    collection_name="faq",
    vectors_config=VectorParams(size=len(vectors[0]), distance=Distance.COSINE)
)

# 🔥 CRITICAL: Create payload index for 'course'
client.create_payload_index(
    collection_name="faq",
    field_name="course",
    field_schema="keyword"
)

# Create point list
points = [
    PointStruct(
        id=i,
        vector=vectors[i],
        payload={
            "id": doc["id"],  # important: used for ground_truth matching
            "course": doc["course"],
            "question": doc["question"],
            "text": doc["text"]
        }
    )
    for i, doc in enumerate(documents)
]

# Upload points
client.upsert(collection_name="faq", points=points)

/var/folders/h2/nzdrmmqs66x13b1h4f0882fh0000gn/T/ipykernel_48825/3469587306.py:8: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(
/var/folders/h2/nzdrmmqs66x13b1h4f0882fh0000gn/T/ipykernel_48825/3469587306.py:14: UserWarning: Payload indexes have no effect in the local Qdrant. Please use server Qdrant if you need payload indexes.
  client.create_payload_index(


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [21]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

def search_function(q):
    vec = model.encode([q["question"]])[0]
    
    hits = client.search(
        collection_name="faq",
        query_vector=vec,
        limit=5,
        query_filter=Filter(
            must=[
                FieldCondition(
                    key="course",
                    match=MatchValue(value=q["course"])
                )
            ]
        ),
        with_payload=True
    )

    return [hit.payload for hit in hits]

In [22]:
result = evaluate(ground_truth, search_function)
print(result)

  0%|          | 0/4627 [00:00<?, ?it/s]/var/folders/h2/nzdrmmqs66x13b1h4f0882fh0000gn/T/ipykernel_48825/1191004656.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(
100%|██████████| 4627/4627 [01:27<00:00, 53.13it/s]

{'hit_rate': 0.21158417981413444, 'mrr': 0.14293638786830937}


## Q5. Cosine simiarity

In the second part of the module, we looked at evaluating
the entire RAG approach. In particular, we looked at 
comparing the answer generated by our system with the actual
answer from the FAQ.

One of the ways of doing it is using the cosine similarity. 
Let's see how to calculate it.

Cosine similarity is a dot product between two normalized vectors.
In geometrical sense, it's the cosine of the angle between
the vectors. Look up "cosine similarity geometry" if you want to
learn more about it.

For us, it means that we need two things:

- First, we normalize each of the vectors
- Then, compute the dot product

So, we get this:

In [ ]:
# Cosine Similarity Function
import numpy as np

def cosine(u, v):
    u_norm = np.sqrt(u.dot(u))
    v_norm = np.sqrt(v.dot(v))
    return u.dot(v) / (u_norm * v_norm)

Now let's use this function to compute the
A->Q->A cosine similarity.

We will use the results from [our gpt-4o-mini evaluations](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/03-evaluation/rag_evaluation/data/results-gpt4o-mini.csv):

In [ ]:
# Load Results CSV
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
results_url = url_prefix + 'rag_evaluation/data/results-gpt4o-mini.csv'

df_results = pd.read_csv(results_url)
df_results.head(2)

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp


In [ ]:
# Create TF-IDF + SVD Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)

In [ ]:
# Fit the Pipeline
pipeline.fit(
    df_results["answer_llm"] + " " +
    df_results["answer_orig"] + " " +
    df_results["question"]
)

,steps,"[('tfidfvectorizer', ...), ('truncatedsvd', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


Now use the `transform` methon of the pipeline to create the embeddings and calculate the cosine similarity between each
pair.

What's the average cosine?

In [ ]:
# Compute Cosine Similarity for Each Row
similarities = []

for _, row in df_results.iterrows():
    v_llm = pipeline.transform([row["answer_llm"]])[0]
    v_orig = pipeline.transform([row["answer_orig"]])[0]
    sim = cosine(v_llm, v_orig)
    similarities.append(sim)

average_cosine = np.mean(similarities)
print("Average cosine similarity:", average_cosine)

Average cosine similarity: 0.8415841233490402


## Q6. Rouge

And alternative way to see how two texts are similar is ROUGE. 

This is a set of metrics that compares two answers based on the overlap of n-grams, word sequences, and word pairs.

It can give a more nuanced view of text similarity than just cosine similarity alone.

We don't need to implement it ourselves, there's a python package for it:


In [34]:
# !pip install rouge
!pip list | grep rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


rouge                   1.0.1


(The latest version at the moment of writing is `1.0.1`)

Let's compute the ROUGE score between the answers at the index 10 of our dataframe (`doc_id=5170565b`)

In [33]:
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
results_url = url_prefix + 'rag_evaluation/data/results-gpt4o-mini.csv'
df_results = pd.read_csv(results_url)

In [36]:
# ROUGE Example for One Row
from rouge import Rouge

rouge_scorer = Rouge()

# Check the 10th row (index 10)
r = df_results.iloc[10]

scores = rouge_scorer.get_scores(r.answer_llm, r.answer_orig)[0]
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

There are three scores: `rouge-1`, `rouge-2` and `rouge-l`, and precision, recall and F1 score for each.

* `rouge-1` - the overlap of unigrams,
* `rouge-2` - bigrams,
* `rouge-l` - the longest common subsequence

For the 10th document, Rouge-1 F1 score is 0.45

Let's compute it for the pairs in the entire dataframe.
What's the average Rouge-1 F1?

In [37]:
print(scores["rouge-1"]["f"])

0.45454544954545456
